In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import math
import numpy as np
import pandas as pd
import glob
import time

from sklearn import linear_model, model_selection, metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import OneClassSVM
from sklearn import model_selection, metrics
from scipy import stats # for statistical tests

import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive

plt.style.use('seaborn')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
# Get the dataset
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
data = pd.read_csv('gdrive/My Drive/cleaned_data.csv', header=0, index_col=None)
data.head()

,Unnamed: 0,Reviews,Restaurant
0,0,['gute hamburger leckeres pale ale und tolle c...,TFDB
1,3,"['burger was ok not fantastic', ' place was e...",TFDB
2,4,['one of the better burgers ive had the bread ...,TFDB
3,5,"['diner time and date 8 october 2021 1pm', ' m...",TFDB
4,6,['excellent burgers with a taste of each ingre...,TFDB


In [5]:
reviews = data['Reviews']

In [6]:
reviews

0      ['gute hamburger leckeres pale ale und tolle c...
1      ['burger was ok not fantastic', '  place was e...
2      ['one of the better burgers ive had the bread ...
3      ['diner time and date 8 october 2021 1pm', ' m...
4      ['excellent burgers with a taste of each ingre...
                             ...                        
932    ['fresh and sweet lobster roll but thought it ...
933                              ['good place to chill']
934                           ['great food very casual']
935    ['steaks are pretty good and i enjoyed the fre...
936                                ['steak and lobster']
Name: Reviews, Length: 937, dtype: object

In [13]:
reviews.to_numpy()

array(["['gute hamburger leckeres pale ale und tolle cocktails']",
       "['burger was ok not fantastic', '  place was extremely hot even though i was there at night room temperature was 32 degrees not 25']",
       "['one of the better burgers ive had the bread is soft and well toasted and the meat is thick and flavorful']",
       "['diner time and date 8 october 2021 1pm', ' my wife and i popped by on the way through errands and oh boy it was wonderful', ' wife had no', '1 and i had the no', '2', ' it was a nice touch that they had 2 servicing sizes smaller size for the discerning ']",
       "['excellent burgers with a taste of each ingredients and come at an affordable price this can come with tasty sides onion rings are dope and on top of this a good service ']",
       "['great burgers that reminds you of la', ' the only thing missing is the perfect weather', ' fantastic char to the patty and the onion rings were nicely done']",
       "['came to try on a discount', ' interesti

In [32]:
arr = []
for i in reviews:
  i = i.replace("[","")
  i = i.replace("]","")
  i = i.replace("'","")
  words = nltk.word_tokenize(i)
  for j in words:
    if j not in arr:
      arr.append(j)
  #words = nltk.word_tokenize(sentence)
  #new_words= [word for word in words if word.isalnum()]

In [33]:
arr

['gute',
 'hamburger',
 'leckeres',
 'pale',
 'ale',
 'und',
 'tolle',
 'cocktails',
 'burger',
 'was',
 'ok',
 'not',
 'fantastic',
 ',',
 'place',
 'extremely',
 'hot',
 'even',
 'though',
 'i',
 'there',
 'at',
 'night',
 'room',
 'temperature',
 '32',
 'degrees',
 '25',
 'one',
 'of',
 'the',
 'better',
 'burgers',
 'ive',
 'had',
 'bread',
 'is',
 'soft',
 'and',
 'well',
 'toasted',
 'meat',
 'thick',
 'flavorful',
 'diner',
 'time',
 'date',
 '8',
 'october',
 '2021',
 '1pm',
 'my',
 'wife',
 'popped',
 'by',
 'on',
 'way',
 'through',
 'errands',
 'oh',
 'boy',
 'it',
 'wonderful',
 'no',
 '1',
 '2',
 'a',
 'nice',
 'touch',
 'that',
 'they',
 'servicing',
 'sizes',
 'smaller',
 'size',
 'for',
 'discerning',
 'excellent',
 'with',
 'taste',
 'each',
 'ingredients',
 'come',
 'an',
 'affordable',
 'price',
 'this',
 'can',
 'tasty',
 'sides',
 'onion',
 'rings',
 'are',
 'dope',
 'top',
 'good',
 'service',
 'great',
 'reminds',
 'you',
 'la',
 'only',
 'thing',
 'missing',
 'p

In [34]:
tagged = nltk.pos_tag(arr)

In [35]:
tagged

[('gute', 'NN'),
 ('hamburger', 'NN'),
 ('leckeres', 'VBZ'),
 ('pale', 'JJ'),
 ('ale', 'JJ'),
 ('und', 'NN'),
 ('tolle', 'NN'),
 ('cocktails', 'NNS'),
 ('burger', 'NN'),
 ('was', 'VBD'),
 ('ok', 'JJ'),
 ('not', 'RB'),
 ('fantastic', 'JJ'),
 (',', ','),
 ('place', 'NN'),
 ('extremely', 'RB'),
 ('hot', 'JJ'),
 ('even', 'RB'),
 ('though', 'IN'),
 ('i', 'NN'),
 ('there', 'RB'),
 ('at', 'IN'),
 ('night', 'NN'),
 ('room', 'NN'),
 ('temperature', 'NN'),
 ('32', 'CD'),
 ('degrees', 'NNS'),
 ('25', 'CD'),
 ('one', 'CD'),
 ('of', 'IN'),
 ('the', 'DT'),
 ('better', 'JJR'),
 ('burgers', 'NNS'),
 ('ive', 'JJ'),
 ('had', 'VBD'),
 ('bread', 'VBN'),
 ('is', 'VBZ'),
 ('soft', 'JJ'),
 ('and', 'CC'),
 ('well', 'RB'),
 ('toasted', 'JJ'),
 ('meat', 'NN'),
 ('thick', 'JJ'),
 ('flavorful', 'JJ'),
 ('diner', 'NN'),
 ('time', 'NN'),
 ('date', 'NN'),
 ('8', 'CD'),
 ('october', 'NN'),
 ('2021', 'CD'),
 ('1pm', 'CD'),
 ('my', 'PRP$'),
 ('wife', 'NN'),
 ('popped', 'VBN'),
 ('by', 'IN'),
 ('on', 'IN'),
 ('way', 'NN

In [39]:
tagged = pd.DataFrame(tagged)

In [40]:
tagged

,0,1
0,gute,NN
1,hamburger,NN
2,leckeres,VBZ
3,pale,JJ
4,ale,JJ
...,...,...
2587,severely,RB
2588,crab,VBP
2589,cracker,NN
2590,worn,NN


In [42]:
tagged.rename(columns={0:'word',1:'tag'}, inplace=True)

In [43]:
from google.colab import files
tagged.to_csv('tagged.csv', index=False) 
files.download('tagged.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>